In [204]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import warnings
import os
warnings.simplefilter("ignore")

In [205]:
pwd = os.getcwd()
df_train  = pd.read_csv(os.path.basename(pwd + '/dc_ipads.csv'))
df_signal = pd.read_csv(os.path.basename(pwd + '/dc_signal_strength.csv'))
df_location = pd.read_csv(os.path.basename(pwd + '/dc_ap_locations.csv'))
df_test = pd.read_csv(os.path.basename(pwd + '/dc_test_data.csv'))
df_train.head()

,iPad Name,iPad ID,iPad x,iPad y,Floor
0,Big Sur,0452f3638d1a,3.0,49.2,3
1,Lake Shasta,0452f36388d5,10.0,40.8,3
2,Santa Beach,0452f3670522,20.3,40.8,3
3,Pismo Beach,0452f3640b3e,21.1,30.0,3
4,Malibu,0452f3696eed,24.0,40.8,3


In [206]:
df_signal.head()

,iPad ID,Access Point ID,Signal Strength
0,0452f3638d1a,b45d50f656d0,43
1,0452f3638d1a,b45d50f656d0,43
2,0452f3638d1a,b45d50f656d0,43
3,0452f3638d1a,b45d50f656d0,43
4,0452f3638d1a,b45d50f656d0,43


In [207]:
# df_signal['Signal Strength'].value_counts()

In [208]:
df_merge = pd.merge(df_signal, df_location)
df = pd.merge(df_train,df_merge)
df.head()

,iPad Name,iPad ID,iPad x,iPad y,Floor,Access Point ID,Signal Strength,Access Point x,Access Point y
0,Big Sur,0452f3638d1a,3.0,49.2,3,b45d50f656d0,43,2.725522,43.234657
1,Big Sur,0452f3638d1a,3.0,49.2,3,b45d50f656d0,43,2.725522,43.234657
2,Big Sur,0452f3638d1a,3.0,49.2,3,b45d50f656d0,43,2.725522,43.234657
3,Big Sur,0452f3638d1a,3.0,49.2,3,b45d50f656d0,43,2.725522,43.234657
4,Big Sur,0452f3638d1a,3.0,49.2,3,b45d50f656d0,43,2.725522,43.234657


In [209]:
df_test.drop(columns = ['iPad Name'],inplace = True)
df_test.head()

,iPad ID,Access Point ID,Signal Strength,Access Point x,Access Point y
0,0452f360ef9f,b45d50f65110,19,20.025663,53.434486
1,0452f360ef9f,b45d50f806d0,29,32.063740,53.824329
2,0452f360ef9f,b45d50f81bb0,9,48.208385,55.907330
3,0452f36c3d9d,b45d50f64bd0,20,89.621559,29.453739
4,0452f36c3d9d,b45d50f64ed0,39,76.194511,29.509821


In [210]:
df['Floor'].value_counts()

3    20684
Name: Floor, dtype: int64

In [211]:
df.drop(columns = ['Floor','iPad Name'],inplace = True)

In [212]:
df['iPad ID'] = df['iPad ID'].astype('category')
df['Access Point ID'] = df['Access Point ID'].astype('category')

df_test['iPad ID'] = df_test['iPad ID'].astype('category')
df_test['Access Point ID'] = df_test['Access Point ID'].astype('category')


In [213]:
df['iPad ID'].value_counts()

0452f36d4491    3222
0452f36d447c    2258
0452f36b2e85    1902
0452f36388d5    1853
0452f3696eed    1323
0452f3638d1a    1259
0452f36b08f1    1192
0452f36b0e74    1112
0452f36bdd3b    1041
0452f3619976    1006
0452f36b0df9     924
0452f3670522     919
0452f36af6e6     854
0452f36c37d6     823
e8b2ac05843d     547
0452f36b0de7     449
Name: iPad ID, dtype: int64

In [214]:
#encoding all the categorical variables:
le = LabelEncoder()

df["iPad ID"] = le.fit_transform(df["iPad ID"])
df["Access Point ID"] = le.fit_transform(df["Access Point ID"])

df_test["iPad ID"] = le.fit_transform(df_test["iPad ID"])
df_test["Access Point ID"] = le.fit_transform(df_test["Access Point ID"])

In [215]:
df.head()

,iPad ID,iPad x,iPad y,Access Point ID,Signal Strength,Access Point x,Access Point y
0,2,3.0,49.2,16,43,2.725522,43.234657
1,2,3.0,49.2,16,43,2.725522,43.234657
2,2,3.0,49.2,16,43,2.725522,43.234657
3,2,3.0,49.2,16,43,2.725522,43.234657
4,2,3.0,49.2,16,43,2.725522,43.234657


In [216]:
df.dtypes

iPad ID              int32
iPad x             float64
iPad y             float64
Access Point ID      int32
Signal Strength      int64
Access Point x     float64
Access Point y     float64
dtype: object

In [217]:
columns = ['iPad x', 'iPad y']
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
y_train = df[['iPad x', 'iPad y']]
X_train = df.drop(columns = columns)

In [218]:
model = MultiOutputRegressor(Ridge(random_state=1)).fit(X_train, y_train)

In [219]:
predict = model.predict(df_test)

In [220]:
df_predict = pd.DataFrame(predict, columns = ['iPad x', 'iPad y'])

In [221]:
dftest = pd.read_csv(os.path.basename(pwd + '/dc_test_data.csv'))

In [222]:
df_predict['iPad ID'] = dftest['iPad ID']

In [224]:
df_predict

,iPad x,iPad y,iPad ID
0,11.317440,43.069048,0452f360ef9f
1,17.319991,43.987900,0452f360ef9f
2,23.365015,41.465193,0452f360ef9f
3,54.752623,26.061983,0452f36c3d9d
4,49.774088,29.443473,0452f36c3d9d
5,44.379874,30.348552,0452f36c3d9d
6,12.205132,32.372477,0452f3641b87
7,21.255027,30.220625,0452f3641b87
8,47.499660,26.289000,0452f3641b87
9,16.566527,31.679837,0452f3641b87


In [225]:
df_predict.to_csv(os.path.basename(pwd +'/final_resul.csv'))